In [1]:
import pandas as pd
import geopandas as gpd
import vaex
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

# 400 meter coordinate accuracy
# 1923 earliest year

In [2]:
df_essential_path = input("Enter the file path: ")

df_essential = vaex.open(df_essential_path)

In [ ]:
df_essential.describe()

In [4]:
df_essential = df_essential.dropna(
    column_names=[
        "decimalLatitude",
        "decimalLongitude",
        "coordinateUncertaintyInMeters",
        "geodeticDatum",
        "species",
        "date_year",
        "basisOfRecord",
    ]
)


In [ ]:
df_essential.describe()


In [ ]:
df_essential.geodeticDatum.value_counts()

In [5]:
df_essential = df_essential[
    df_essential.geodeticDatum.isin(
        ["EPSG:4326", "EPSG:4326 WGS84", "EPSG: 4326 WGS 84", "WGS84_EPSG:4326"]
    )
]


In [ ]:
# find unique years in df_essential
years = df_essential["date_year"].unique()

In [6]:
# for some reason have to extract to new df to continue filtering
df_essential = df_essential.extract()

In [7]:
df_essential["date_year"] = df_essential["date_year"].astype("float32")

df_essential["coordinateUncertaintyInMeters"] = df_essential[
    "coordinateUncertaintyInMeters"
].astype("float32")


In [8]:
df_essential = df_essential[df_essential.date_year >= 1923]

In [9]:
max_year = df_essential["date_year"].max()

min_year = df_essential["date_year"].min()


In [10]:
df_essential = df_essential[df_essential.coordinateUncertaintyInMeters <= 400]

In [11]:
max_meter = df_essential["coordinateUncertaintyInMeters"].max()

min_meter = df_essential["coordinateUncertaintyInMeters"].min()

In [ ]:
basis = df_essential["basisOfRecord"].unique()

In [12]:
df_essential = df_essential[
    df_essential.basisOfRecord.isin(
        ["HumanObservation", "MachineObservation", "Occurrence"]
    )
]

In [ ]:
df_essential["basisOfRecord"].value_counts()

In [ ]:
df_essential["flags"].value_counts()

In [13]:
df_essential = df_essential[
    "kingdom",
    "phylum",
    "class",
    "order",
    "family",
    "genus",
    "species",
    "decimalLatitude",
    "decimalLongitude",
    "license",
    "rightsHolder",
]


In [ ]:
df_essential.describe()

In [14]:
species_unique = df_essential.species.unique()

In [15]:
species_values = np.array([len(word.split()) for word in species_unique])

# find the max value in species_values
max_value = np.max(species_values)

In [16]:
# if value at index i in species_values is greater than 2, add the value at the same index in species_unique to a new list
species_2 = []
for i in range(len(species_values)):
    if species_values[i] > 2:
        species_2.append(species_unique[i])

In [17]:
# remove values from species that are greater than 2

for i in range(len(species_2)):
    df_essential = df_essential[df_essential.species != species_2[i]]

In [18]:
df_essential = df_essential.extract()


In [19]:
# recheck
species_unique = df_essential["species"].unique()


In [20]:
species_values = np.array([len(word.split()) for word in species_unique])


max_value = np.max(species_values)


In [ ]:
df_essential["license"].value_counts()

In [22]:
license = df_essential["license"].unique()


In [32]:
license_removal = [
    "Copyright Frank Garita Alpízar. No use without permission.",
    "Copyright Ma. Eugenia Rodríguez Vázquez. No use without permission.",
    "Copyright Whales of Guerrero (contact: Katherina Audley). No use without permission.",
    "Copyright Rémi Bigonneau. No use without permission.",
    "Copyright Nico Ransome. No use without permission.",
    "Copyright Russian Cetacean Habitat Project. No use without permission.",
    "Copyright Jodi Frediani. No use without permission.",
    "Copyright Adam A. Pack. No use without permission.",
    "Copyright Tory Kallman. No use without permission.",
    "Copyright Tasli Shaw. No use without permission.",
    "Copyright Amy Kennedy. No use without permission.",
]

In [33]:
for i in range(len(license_removal)):
    df_essential = df_essential[df_essential.license != license_removal[i]]


In [34]:
df_essential = df_essential.extract()


In [35]:
df_essential["license"].value_counts()


missing                                                       6250826
http://creativecommons.org/licenses/by-nc/4.0/                3282348
http://creativecommons.org/licenses/by/4.0/                    788852
http://creativecommons.org/licenses/by/4.0/legalcode           326587
https://creativecommons.org/publicdomain/zero/1.0               54423
http://creativecommons.org/publicdomain/zero/1.0/legalcode      44681
https://creativecommons.org/licenses/by-sa/4.0                   2794
https://creativecommons.org/licenses/by/4.0                      1904
https://creativecommons.org/licenses/by-nc-sa/4.0                 365
https://creativecommons.org/licenses/by-nc/4.0                    116
https://creativecommons.org/licenses/by-nc-nd/4.0                 114
https://creativecommons.org/licenses/by-nd/4.0                      1
dtype: int64

In [36]:
df_essential.describe()


,kingdom,phylum,class,order,family,genus,species,decimalLatitude,decimalLongitude,license,rightsHolder
data_type,string,string,string,string,string,string,string,float32,float32,string,string
count,10753011,10718528,10594180,10684780,10748831,10753011,10753011,10753011,10753011,4502185,82184
NA,0,34483,158831,68231,4180,0,0,0,0,6250826,10670827
mean,--,--,--,--,--,--,--,48.16344879339813,-23.806153108599656,--,--
std,--,--,--,--,--,--,--,16.584034,52.797058,--,--
min,--,--,--,--,--,--,--,-77.699997,-179.998001,--,--
max,--,--,--,--,--,--,--,88.963799,179.998001,--,--


In [37]:
# read cleaned df_essential into a parquet file where user can input the file path
df_clean_path = input("Enter the file path: ")
df_essential.export_parquet(df_clean_path)